In [1]:
%matplotlib inline

In [2]:
import subprocess
import glob
from Bio import Phylo
import os

In [3]:
def bash_command(cmd):
    return subprocess.Popen(cmd, shell=True, executable='/bin/bash')

# Re-write tree / fastas without periods in names

In [4]:
tree = Phylo.read('../Data/Tree_files/Order_Enterobacterales_Mixed/RAxML_bestTree.MixedSet108seqsFLAT', 'newick')
for node in tree.get_terminals():
    node.name = node.name.replace('.', '_')
Phylo.write(tree, './mixed_tree_no_periods.newick', 'newick');

# Run HYPHY on all

In [5]:
command = '/Users/adamhockenberry/workspace/hyphy/HYPHYMP LIBPATH=`pwd`/../../../../workspace/hyphy/res ../../../../workspace/hyphy/res/TemplateBatchFiles/relative_nucleotide_rates.bf '
arguments = '(echo /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/temp.fa; echo /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/mixed_tree_no_periods.newick; echo 1) | '

In [6]:
print(arguments + command)

(echo /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/temp.fa; echo /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/mixed_tree_no_periods.newick; echo 1) | /Users/adamhockenberry/workspace/hyphy/HYPHYMP LIBPATH=`pwd`/../../../../workspace/hyphy/res ../../../../workspace/hyphy/res/TemplateBatchFiles/relative_nucleotide_rates.bf 


In [7]:
for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/*_4*.mfasta'):
    print(infile_name)
    gene_name = infile_name.split('/')[-1].rstrip('.mfasta')
    rate_file_name = '../Results/Order_Enterobacterales_Mixed/Hyphy_GTR/{}.site-rates.json'.format(gene_name)
    if os.path.isfile(rate_file_name) == True:
        continue    
        
    infile = open(infile_name, 'r').readlines()
    with open('temp.fa', 'w') as outfile:
        for line in infile:
            if line[0] == '>':
                outfile.write(line.replace('.', '_'))
            else:
                outfile.write(line)
    process = bash_command(arguments + command)
    process.communicate()
    
    move_command = 'mv ./temp.fa.site-rates.json {}'.format(rate_file_name)
    process = bash_command(move_command)
    process.communicate()

../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4000.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4007.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4010.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4018.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4024.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4028.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4029.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4031.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4032.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4034.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/VBIEscCol129921_4036.mfasta
../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas

In [ ]:
for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/*_1*.mfasta'):
    gene_name = infile_name.split('/')[-1].rstrip('.mfasta')
    print(infile_name)
    rate_file_name = '/Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Results/Order_Enterobacterales_Mixed/Hyphy_JC/{}.rates'.format(gene_name)
    if os.path.isfile(rate_file_name) == True:
        continue

    infile = open(infile_name, 'r').readlines()
    with open('temp.fa', 'w') as outfile:
        for line in infile:
            if line[0] == '>':
                outfile.write(line.replace('.', '_'))
            else:
                outfile.write(line)
                
    command = 'cat temp.fa mixed_tree_no_periods.newick > temp_hyphy_align.txt'
    process = bash_command(command)
    process.communicate()
    with open('./proteinER-master/src/temp_hyphy_setup.txt', 'w') as outfile:
        outfile.write('INFILE = /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/temp_hyphy_align.txt\n')
        outfile.write('OUTFILE = {}'.format(rate_file_name))
    command = 'HYPHYMP CPU=2 /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/proteinER-master/src/fitrates_JC_equal_NT.bf <<< temp_hyphy_setup.txt'
    process = bash_command(command)
    process.communicate()